# Скачиванием репозиторий и данные

Чтобы скачать репозиторий нужно выполнить команду

```
git clone https://github.com/Wunder-Fund/wunder_challenge ~/wunder_challenge
```

После скачивания репозитория нужно скачать данные для обучения по [ссылке](https://drive.google.com/file/d/1VoZfPUzDYD8afzowLrQfL9G2iMwhcajM/view).  

Распакованные npz файлы нужно поместить в папку `~/wunder_challenge/data/`.


### В конкурсе используется три однородных набора данных

- **train** — доступен участникам для обучения, в нём известны классы моментов `Y`
    - Для удобства участников тренировочный набор доступен в двух вариантах
        - **train.npz**, содержащий тренировочную выборку целиком
        - **train_small_A.npz**, **train_small_B.npz**, **train_small_C.npz** — та же самая выборка, разбитая на три части. Этот вариант удобно использовать для кросс-валидации
- **test** —  используется для составления публичного лидерборда до окончания конкурса
- **validation** — используется для составления финального лидерборда

# Анализ данных 

### Все сообщения в файлах имеют структуру Event

In [8]:
from collections import namedtuple

EventBody = namedtuple('EventBody', 
                   ['time', 'id', 'action', 'type', 'side', 'price', 'amount', 'is_snapshot', 'Y'])

class Event(EventBody):
    @property
    def need_prediction(self):
        return self.Y >= 0

### Описание полей Event

* **time** - время события с точностью 1 микросекунда
* **action** - событие которое произошло 
  * action = 0: удаление ордера
  * action = 1: добавление ордера
  * action = 2: изменение объёма ордера
  * action = 3: сделка
  * action = 10: означает что предыдущий кусок данных закончен и начинается новый.
* **id** - Номер ордера с которым происходит событие (добавление, удаление, изменение объема)
  * для сделок это номер сделки
* **type** - Тип ордера 
  * type = 1: LIMIT ордер - это ордер который останется в стакане до тех пор пока на него не прийдёт удалние
  * type = 2: IOC ордер - это ордер который сразу же удалится после добавляния в стакан и сведение сделок
* **side** - Сторона ордера для событий action = 0,1,2 и сторона аггресивного ордера для сделки (action = 3)
  * side = 0: ордер на покупку
  * side = 1: ордер на продажу
* **price** - Цена ордера или сделки (action = 3)
* **amount** - Новый объем ордера при добавлении/изменении объема (action = 1,2) или сделки (action = 3)
* **is_snapshot** - флаг означающий из снимка стакана ордер или нет 
  * is_snapshot = True: событие из снимка стакана
  * is_snapshot = False: событие не из снимка стакана
  * Данные записаны следущим образом: 
      * сначала идёт порция событий чтобы построить начальное состояние стакана - снимок стакана
      * затем идут обычный лог событий изменяющих стакан 
* **Y** - целевой класс моментов который надо предсказать для момента для которого он проставлен
  * Y = 0, 1 -- значит для текущего события нужно предсказать класс 0 или 1
  * Y < 0 - на этом моменте не нужно ничего предсказывать
  * На большинстве моментов ничего предсказывать не нужно


### Загрузим данные и посмотрим на них

In [13]:
import numpy as np
# Данные хранятся в сжатом numpy массиве
data = np.load("../data/train_small_A.npz")

# загружаем события
events_raw = data["events"]
events = [Event(*ev) for ev in events_raw[:10000]]
    
def find_first_event(action, is_snapshot, need_prediction):
    '''Находим первое событие с нужными параметрами'''
    found_events = [ev for ev in events 
                    if ev.action == action
                      and ev.is_snapshot == is_snapshot 
                      and ev.need_prediction == need_prediction]
    return found_events[0]

## Примеры различных событий

### Новый кусок данных

In [14]:
# action = 10 - значит начался новый кусок данных
print(events[0])

Event(time=0, id=0, action=10, type=0, side=0, price=0, amount=0, is_snapshot=0, Y=-1)


### Добавление ордера

In [17]:
# action = 1 - добавление ордера LIMIT (type=1) на продажу (side=1) 
# по цене price=5035 и объёмом amount=2.
# Это событие из снимка стакана (is_snapshot = 1)
# Для него ничего предсказывать не нужно (Y < 0)

print(find_first_event(action=1, is_snapshot=1, need_prediction=False))

Event(time=42687740354, id=0, action=1, type=1, side=1, price=5035, amount=2, is_snapshot=1, Y=-1)


### Удаление ордера

In [20]:
# action = 0 - удаление ордера id=3118 на покупку (side=0) по цене price=1078.
# Это событие не из снимка стакана (is_snapshot = 0)
# Для него нужно предсказать что Y=0

print(find_first_event(action=0, is_snapshot=0, need_prediction=True))

Event(time=14170040196, id=3118, action=0, type=1, side=0, price=1078, amount=0, is_snapshot=0, Y=0)


### Изменение объёма ордера

In [23]:
# action = 2 - изменение объема ордера id=3201 на продажу (side=1) 
# по цене price=1208. Новый объём = amount = 1
# Это событие не из снимка стакана (is_snapshot = 0)
# Для него ничего предсказывать не нужно (Y < 0)

print(find_first_event(action=2, is_snapshot=0, need_prediction=False))

Event(time=14156125404, id=3201, action=2, type=1, side=1, price=1208, amount=1, is_snapshot=0, Y=-1)


### Сделка

In [25]:
# action = 3 - это сделка по цене price=1208 на объём amount=1. 
# Сторона агрессивного ордера side=1. 
# Это значит что кто-то продал своим агрессивным ордером тому кто уже стоял в стакане
# Это событие не из снимка стакана (is_snapshot = 0)
# Для него ничего предсказывать не нужно (Y < 0)

print(find_first_event(action=3, is_snapshot=0, need_prediction=False))

Event(time=14154129630, id=3180, action=3, type=0, side=1, price=1208, amount=1, is_snapshot=0, Y=-1)
